In [2]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from build_network import build_and_optimize_network
#from build_network import extract_detailed_results


In [4]:
#load the data file
timeseries_data_full = pd.read_csv('Inputs_GB_LTW_2030_baseline.csv')
start_date = '2030-01-01 00:00:00'
datetime_index = pd.date_range(start=start_date, periods=len(timeseries_data_full), freq='h')

# Set the new DatetimeIndex and remove the old 'Period' column
timeseries_data_full.index = datetime_index
timeseries_data_full = timeseries_data_full.drop(columns=['Period'])
timeseries_data_full.head(5)

,Demand,PV_Z1,PV_Z2,PV_Z3,PV_Z4,PV_Z5,PV_Z6,PV_Z7,PV_Z8,PV_Z9,...,Z3_wave,Z8_wave,Z9_wave,Z1_tidal,Z3_tidal,Z5_tidal,Z7_tidal,Z9_tidal,DSR_min,DSR_max
2030-01-01 00:00:00,26276.86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.847830,0.572871,0.537692,0.207735,0.009276,0.025925,0.011799,0.019891,-1.0,1.0
2030-01-01 01:00:00,25831.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.847830,0.572546,0.533920,0.035931,0.108594,0.176336,0.195456,0.212510,-1.0,1.0
2030-01-01 02:00:00,24952.96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.845383,0.567793,0.527585,0.205646,0.395722,0.457483,0.598944,0.554929,-1.0,1.0
2030-01-01 03:00:00,24178.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.837562,0.558284,0.522157,0.669624,0.522340,0.564950,0.749965,0.594264,-1.0,1.0
2030-01-01 04:00:00,23628.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.827409,0.548857,0.516379,0.787130,0.407776,0.506456,0.539666,0.405528,-1.0,1.0


In [5]:
# --- Data Dictionaries for Renewables + Storage Scenario ---

# add carriers that are relevant to the model
relevant_carriers = [
    "onwind", "offwind", "tidal",
    "battery", "hydroelectric_ph", "DSR", "load_shed", "AC"
]

# Base marginal costs for each technology type, in GBP/MWh.
base_marginal_costs = {
    "onwind": 0, "offwind": 0, "tidal": 0,
    "battery": 1, "hydroelectric_ph": 60, "DSR": 30, "load_shed": 10e6
}

# Base capital costs for each technology type, in GBP/MW
base_capital_costs = {
    "onwind": 1.2e6,
    "offwind": 1.81e6,     
    "tidal": 4.9e6,        
    "battery": 0.425e6,       
    "hydroelectric_ph": 2.0e6,
    "AC": 0.119e6
}


operational_penalties = {
    "onwind": {
        # A mid-range penalty reflecting the continuous noise and visual impact on local communities and birds.
        "biodiversity & noise": 5,
        # A moderately high penalty due to the large land area turbines occupy, even with some multi-use possible.
        "land use": 6
    },
    "offwind": {
        # A significant penalty due to the operational noise from turbines affecting sensitive marine mammals and ecosystems.
        "biodiversity & noise": 6,
        # A very low penalty as the only land use is for a small onshore substation.
        "land use": 2
    },
    "tidal": {
        # A very high penalty, reflecting the major disruption that operating turbines cause in ecologically concentrated tidal zones.
        "biodiversity & noise": 8,
        # A very low penalty, similar to offshore wind, as the land footprint is minimal.
        "land use": 2
    },
    "battery": {
        # A low penalty, representing the constant but localized hum from cooling systems and inverters.
        "biodiversity & noise": 2,
        # A low penalty due to the small, contained physical footprint of the facility.
        "land use": 2
    },
    "hydroelectric_ph": {
        # A significant penalty for the major and permanent disruption to river ecosystems and fish migration from daily operations.
        "biodiversity & noise": 6,
        # A high penalty reflecting the permanent loss of land from flooding it for two large reservoirs.
        "land use": 7
    },
}
# Each penalty is an independent score between 0 and 1.
# Keys have been renamed for clarity in plots and analysis.
construction_penalties = {
    "onwind": {
        "ghg": 0.15,                  # Low GHG: Based on concrete for foundation and steel for tower. Nudged up to be more competitive with offshore.
        "biodiversity & noise": 0.18, # Low impact: Construction noise is temporary and localized. Nudged up to be closer to offshore.
        "land use": 0.90,             # High impact: Occupies a large area, though the land can often be multi-use (e.g., farming).
        "grid upgrades": 0.60,        # Very High impact: Reflects the high system cost of managing intermittency with storage/backup.
        "reliability": 0.85           # High penalty: Highly variable output dependent on weather.
    },
    "offwind": {
        "ghg": 0.20,                  # Medium GHG: Larger turbines and subsea foundations are material-intensive. Lowered to be more competitive.
        "biodiversity & noise": 0.20, # Medium impact: Pile driving for foundations is very disruptive to marine life.
        "land use": 0.25,             # Low-Medium impact: Requires significant onshore substation and port facilities for construction and maintenance.
        "grid upgrades": 0.70,        # High impact: High cost of subsea cables combined with the need for grid support for intermittency.
        "reliability": 0.65           # Medium-High penalty: More consistent and stronger than onshore, but still variable.
    },
    "tidal": {
        "ghg": 0.40,                  # High GHG: Extremely material-intensive (steel/concrete) due to immature tech and harsh marine environment.
        "biodiversity & noise": 0.45, # High impact: Major construction disruption in sensitive and concentrated tidal ecosystems.
        "land use": 0.10,             # Very low impact: Minimal land use, primarily for an onshore substation and grid connection.
        "grid upgrades": 0.10,        # Very Low impact: Predictability means far less need for costly grid support, making it the best option for grid stability.
        "reliability": 0.10           # Very low penalty: Not constant, but 100% predictable based on tides, making it highly reliable for the grid.
    },
    "battery": {
        "ghg": 0.30,                  # Medium-High GHG: Dominated by energy-intensive mining and processing of raw materials (lithium, cobalt).
        "biodiversity & noise": 0.05, # Very low impact: Standard, temporary construction site noise.
        "land use": 0.15,             # Low impact: A concentrated, small footprint for the facility itself.
        "grid upgrades": 0.20,        # Medium impact: Requires robust substation upgrades to handle high-power, two-way energy flow (charging and discharging).
        "reliability": 0.05           # Very low penalty: A primary solution to intermittency; output is dispatchable and highly reliable.
    },
    "hydroelectric_ph": {
        "ghg": 0.25,                  # Medium GHG: Involves massive amounts of concrete for dam and reservoir construction.
        "biodiversity & noise": 0.30, # High impact: A major civil engineering project with significant, long-term ecosystem disruption.
        "land use": 0.95,             # Very high impact: Involves the permanent and total consumption of land by flooding it for two large reservoirs.
        "grid upgrades": 0.25,        # Medium impact: Requires a very high-capacity connection to the grid to handle large power flows.
        "reliability": 0.05           # Very low penalty: A primary solution to intermittency; dispatchable and very reliable output.
    },
}


Run the below cell to see how performance scales with the number of time-series snapshots

In [ ]:
sample_sizes_to_test = [200, 1000, 2000, 5000, 8760]

# Use a single, simple penalty scenario for this timing test.
base_test_weights = {
    "ghg": 2,
    "biodiversity & noise": 2,
    "land use": 2,
    "grid upgrades": 2,
    "reliability": 2
}
scenario_name_template = "timing_test_{n}_snapshots"

print("--- Starting Computation Time Analysis ---")

# Loop through each sample size
for n_snapshots in sample_sizes_to_test:
    print(f"\n--- Testing with {n_snapshots} snapshots ---")

    # Subsample the full timeseries data to the current size
    indices = np.linspace(0, len(timeseries_data_full) - 1, n_snapshots, dtype=int)
    subsampled_data = timeseries_data_full.iloc[indices]

    # Record the start time
    start_time = time.time()

    try:
        network = build_and_optimize_network(
            scenario_name_template.format(n=n_snapshots),
            base_test_weights,
            subsampled_data,
            relevant_carriers,
            base_marginal_costs,
            base_capital_costs,
            operational_penalties,
            construction_penalties
        )

        # Record the end time and calculate duration
        end_time = time.time()
        duration_seconds = end_time - start_time

        # Print the result
        print(f"---> Run with {n_snapshots} snapshots took {duration_seconds:.2f} seconds.")

    except Exception as e:
        # If the optimization fails, print failure message
        print(f"---> Run with {n_snapshots} snapshots failed: {e}")

print("\n--- Computation Time Analysis Finished ---")

--- Starting Computation Time Analysis ---

--- Testing with 200 snapshots ---

--- Running Scenario: timing_test_200_snapshots ---


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 8/8 [00:00<00:00, 436.85it/s]
INFO:linopy.io: Writing time: 0.18s


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-2_5y1_le has 38845 rows; 17045 cols; 76230 nonzeros
Coefficient ranges:
  Matrix [1e-03, 4e+01]
  Cost   [4e+02, 4e+08]
  Bound  [0e+00, 0e+00]
  RHS    [2e+01, 3e+04]
Presolving model
19400 rows, 17045 cols, 56785 nonzeros  0s
Dependent equations search running on 4786 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
19372 rows, 17017 cols, 56701 nonzeros  0s
Presolve : Reductions: rows 19372(-19473); columns 17017(-28); elements 56701(-19529)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1800(2.96647e+07) 0s


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 17045 primals, 38845 duals
Objective: 1.82e+12
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


      16591     1.8181590288e+12 Pr: 0(0); Du: 0(8.6602e-08) 1s
      16591     1.8181590288e+12 Pr: 0(0); Du: 0(8.6602e-08) 1s
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-2_5y1_le
Model status        : Optimal
Simplex   iterations: 16591
Objective value     :  1.8181590288e+12
P-D objective error :  1.0742322146e-15
HiGHS run time      :          1.43
Writing the solution to /private/var/folders/h9/ylqhszz50d9dcx3w0hkvc7l40000gp/T/linopy-solve-qxmtevah.sol
Optimization status for timing_test_200_snapshots: ok
---> Run with 200 snapshots took 3.29 seconds.

--- Testing with 1000 snapshots ---

--- Running Scenario: timing_test_1000_snapshots ---


...

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-jvsj_kx2 has 970045 rows; 425045 cols; 1905030 nonzeros
Coefficient ranges:
  Matrix [6e-04, 2e+01]
  Cost   [2e+01, 2e+07]
  Bound  [0e+00, 0e+00]
  RHS    [2e+01, 4e+04]
Presolving model
485000 rows, 425045 cols, 1419985 nonzeros  0s
Dependent equations search running on 119986 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.01s (limit = 1000.00s)
484972 rows, 425017 cols, 1419901 nonzeros  0s
Presolve : Reductions: rows 484972(-485073); columns 425017(-28); elements 1419901(-485129)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 45000(1.04279e+08) 1s
      35205     3.8318259972e+10 Pr: 55580(3.79895e+08); Du: 0(6.68695e-08) 6s
      40811     3.8609155523e+10 Pr: 55942(3.48285e+08); Du: 0(6.68695e-08) 12s
      

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 425045 primals, 970045 duals
Objective: 1.93e+12
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


Optimization status for timing_test_5000_snapshots: ok
---> Run with 5000 snapshots took 512.22 seconds.

--- Testing with 8760 snapshots ---

--- Running Scenario: timing_test_8760_snapshots ---


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 8/8 [00:00<00:00, 13.73it/s]
INFO:linopy.io: Writing time: 4.49s


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-b9jolcrt has 1699485 rows; 744645 cols; 3337590 nonzeros
Coefficient ranges:
  Matrix [6e-04, 2e+01]
  Cost   [9e+00, 2e+07]
  Bound  [0e+00, 0e+00]
  RHS    [2e+01, 4e+04]
Presolving model
849720 rows, 744645 cols, 2487825 nonzeros  0s
Dependent equations search running on 210226 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.01s (limit = 1000.00s)
849692 rows, 744617 cols, 2487741 nonzeros  1s
Presolve : Reductions: rows 849692(-849793); columns 744617(-28); elements 2487741(-849849)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 78840(1.65663e+08) 2s
      29558     1.9513380681e+10 Pr: 81370(2.54419e+08); Du: 0(6.90736e-08) 8s
      39915     4.2745272565e+10 Pr: 90415(7.81612e+08); Du: 0(9.90279e-08) 15s
     